Install Dependencies

In [1]:
# Processing libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

#Imports pickle
import pickle

#Library for indicators
!pip install finta
from finta import TA

#Another library for indicators
! pip install --upgrade ta
from ta.utils import dropna
from ta.volatility import BollingerBands
from ta.trend import PSARIndicator
from ta.trend import SMAIndicator
from ta.trend import macd
from ta.momentum import rsi
from ta.volume import OnBalanceVolumeIndicator

  Created wheel for ta: filename=ta-0.10.1-py3-none-any.whl size=28986 sha256=17edb77008d10c574cee8617201f0126e2a0bc9a473ed1557e614d202e0fcd70
  Stored in directory: /Users/esteban/Library/Caches/pip/wheels/fa/f5/2a/22332fca050eb5e48116f8534cd267d50cffcd1e7581606150
Successfully built ta


Selects the dataframe for training

In [3]:
#df = pd.read_csv('/content/drive/MyDrive/Other Projects/USDGBP.csv')
#df = pd.read_csv('/content/drive/MyDrive/Other Projects/USDJPY.csv')
df = pd.read_csv('BTCUSD_day.csv')
#df = pd.read_csv('/content/drive/MyDrive/Other Projects/BTCUSD_1hr.csv')
#df = pd.read_csv('/content/drive/MyDrive/Other Projects/BTCUSD_1min.csv')
#df = pd.read_csv('/content/drive/MyDrive/Other Projects/Download Data - STOCK_US_XNYS_F (1).csv')

df

,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume USD
0,2020-04-10,BTCUSD,7315.25,7315.25,7315.25,7315.25,0.00,0.00
1,2020-04-09,BTCUSD,7369.60,7378.85,7115.04,7315.25,2237.13,16310014.53
2,2020-04-08,BTCUSD,7201.81,7432.23,7152.80,7369.60,2483.60,18138080.27
3,2020-04-07,BTCUSD,7336.96,7468.42,7078.00,7201.81,2333.34,17047120.32
4,2020-04-06,BTCUSD,6775.21,7369.76,6771.01,7336.96,3727.47,26533750.17
...,...,...,...,...,...,...,...,...
1642,2015-10-12,BTCUSD,249.50,249.50,247.60,247.60,38.28,9493.89
1643,2015-10-11,BTCUSD,246.30,249.50,246.30,249.50,12.22,3021.12
1644,2015-10-10,BTCUSD,245.39,246.30,244.60,246.30,12.17,2984.44
1645,2015-10-09,BTCUSD,243.95,249.97,243.60,245.39,30.99,7651.63


Data Cleanup

In [7]:
print(df.dtypes)

#Convert date
#df['Date'] = pd.to_datetime(df['Date'])

#Sort values from past to present
df.sort_values('Date', ascending=True, inplace=True)

#Set date as index
df.set_index('Date', inplace=True)

#Converts volume from string to float
#df.drop('Volume', inplace=True, axis=1)
#df['Volume'] = df['Volume'].apply(lambda x: float(x.replace(",", "")))
#df['Volume'] = df['Volume'].astype(float)
#df = df.drop(columns=['Symbol'])
#df = df.drop(columns=['Unix Timestamp'])
df = df.drop(columns=['Volume USD'])
df = df.rename(columns={'Volume BTC': 'Volume'})

print(df.shape[0])
print(df.dtypes)

df.head()

Symbol     object
Open      float64
High      float64
Low       float64
Close     float64
Volume    float64
dtype: object


KeyError: "None of ['Date'] are in the columns"

Normalization

In [ ]:
'''
df["Open"] = df["Open"] - df["Open"].shift(1)
df["High"] = df["High"] - df["High"].shift(1)
df["Low"] = df["Low"] - df["Low"].shift(1)
df["Close"] = df["Close"] - df["Close"].shift(1)
'''

'\ndf["Open"] = df["Open"] - df["Open"].shift(1)\ndf["High"] = df["High"] - df["High"].shift(1)\ndf["Low"] = df["Low"] - df["Low"].shift(1)\ndf["Close"] = df["Close"] - df["Close"].shift(1)\n'

# 2.1 Add indicators
2.1.1 Install New Dependencies

2.1.3. Calculate SMA, RSI and OBV


In [13]:

#Indicators with TA
#Adds simple moving average
df["sma7"] = SMAIndicator(close=df["Open"], window=7, fillna=True).sma_indicator()
df["sma25"] = SMAIndicator(close=df["Open"], window=25, fillna=True).sma_indicator()
df["sma99"] = SMAIndicator(close=df["Open"], window=99, fillna=True).sma_indicator()

#Adds Bollinger Bands
indicator_bb = BollingerBands(close=df["Open"], window=20, window_dev=2)
df['bb_bbm'] = indicator_bb.bollinger_mavg()
df['bb_bbh'] = indicator_bb.bollinger_hband()
df['bb_bbl'] = indicator_bb.bollinger_lband()

#Adds PSAR Indicator
indicator_psar = PSARIndicator(high=df["Close"], low=df["Low"], close=df["Close"], step=0.02, max_step=2, fillna=True)
df['psar'] = indicator_psar.psar()

# Add Moving Average Convergence Divergence (MACD) indicator
df["MACD"] = macd(close=df["Close"], window_slow=26, window_fast=12, fillna=True)
    
# Add Relative Strength Index (RSI) indicator
df["RSI"] = rsi(close=df["Close"], window=14, fillna=True)

#Indicators using finta
#Adds simple moving average
df["sma40"] = TA.SMA(df, period = 40)

# Add Relative Strength Index (RSI) indicator
df["OBV"] = TA.OBV(df)

#Drops empty spaces
df.fillna(0, inplace=True)

#Drops empty spaces
df.head(50)


/Users/esteban/miniforge3/lib/python3.9/site-packages/ta/trend.py:938: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_up = pd.Series(index=self._psar.index)
/Users/esteban/miniforge3/lib/python3.9/site-packages/ta/trend.py:939: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  self._psar_down = pd.Series(index=self._psar.index)


,Symbol,Open,High,Low,Close,Volume,sma40,OBV,sma7,sma25,sma99,bb_bbm,bb_bbh,bb_bbl,psar,MACD,RSI
Date,,,,,,,,,,,,,,,,,
2015-10-08,BTCUSD,242.50,245.00,242.50,243.95,18.80,0.00000,0.00,242.500000,242.500000,242.500000,0.0000,0.000000,0.000000,243.950000,0.000000,100.000000
2015-10-09,BTCUSD,243.95,249.97,243.60,245.39,30.99,0.00000,30.99,243.225000,243.225000,243.225000,0.0000,0.000000,0.000000,245.390000,0.114872,100.000000
2015-10-10,BTCUSD,245.39,246.30,244.60,246.30,12.17,0.00000,43.16,243.946667,243.946667,243.946667,0.0000,0.000000,0.000000,243.950000,0.276155,100.000000
2015-10-11,BTCUSD,246.30,249.50,246.30,249.50,12.22,0.00000,55.38,244.535000,244.535000,244.535000,0.0000,0.000000,0.000000,244.600000,0.654639,100.000000
2015-10-12,BTCUSD,249.50,249.50,247.60,247.60,38.28,0.00000,17.10,245.528000,245.528000,245.528000,0.0000,0.000000,0.000000,244.600000,0.792146,72.095821
2015-10-13,BTCUSD,247.60,252.87,245.75,250.00,51.71,0.00000,68.81,245.873333,245.873333,245.873333,0.0000,0.000000,0.000000,244.698000,1.082304,79.773530
2015-10-14,BTCUSD,250.00,254.75,250.00,253.00,119.54,0.00000,188.35,246.462857,246.462857,246.462857,0.0000,0.000000,0.000000,244.910080,1.536619,85.240336
2015-10-15,BTCUSD,253.00,258.98,253.00,255.26,197.11,0.00000,385.46,247.962857,247.280000,247.280000,0.0000,0.000000,0.000000,245.395475,2.055337,87.894702
2015-10-16,BTCUSD,255.90,269.98,255.90,263.75,101.13,0.00000,486.59,249.670000,248.237778,248.237778,0.0000,0.000000,0.000000,246.184637,3.115582,92.992831


*2.1*.4. Dump dataset into a pickle file

In [14]:
import pickle
with open("BTCUSD_1day.pickle", 'wb') as f:
      pickle.dump(df,f, pickle.HIGHEST_PROTOCOL)
